## Accessing the Clipboard using the `pyperclip` Library

Often I find myself needing to copy the contents of some variable into some other application.

Sure, you can just run the script/app in debug mode, and do some inspection, or you could even add some code to write what you need to some file, but sometimes, especially if the content is plain text, it's much simpler to use the `pyperclip` library.

The `pyperclip` library is a very basic library to access your local clipboard (and it is cross-platform), for storing and retrieving text - you can't use it for things like files, but as long as it is text, it works wonderfully well.

The library and it's docs are located [here](https://github.com/asweigart/pyperclip).

First you'll need to install `pip` install the library `pyperclip`.

It's also included in the `Pipfile` with this repo, so you can just use `pipenv` to install all the libraries needed by this repo.

Now let's try it out.

In [1]:
import pyperclip

In [2]:
s = "Python rocks!"

In [3]:
pyperclip.copy(s)

And that's it, the text has been copied to your clipboard - go paste somewhere else, and you should see `Python rocks!`.

You can also grab the current clipboard entry (as long as it's basically text):

In [4]:
contents = pyperclip.paste()
print(contents)

Python rocks!


You cannot use `pyperclip` to push other complex objects onto the clipboard, but there are ways around that.

Let's say you want to put the contents of a Python dictionary into the clipboard:

In [5]:
d = {
    'a': 1,
    'b': 2,
    'c': 3
}

In [6]:
try:
    pyperclip.copy(d)
except pyperclip.PyperclipException as ex:
    print(ex)

only str, int, float, and bool values can be copied to the clipboard, not dict


We can get around this limitation by serializing the data to a string representation.

For example, we could use Python's `pickle` to serialize this object:

In [7]:
import pickle

In [8]:
d_pickled = pickle.dumps(d)
print(d_pickled)

b'\x80\x04\x95\x17\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x01a\x94K\x01\x8c\x01b\x94K\x02\x8c\x01c\x94K\x03u.'


Now, we're not done yet, since pickled data is essentially binary data.

One approach here would be to base64 encode this data (so we're only dealing with ASCII characters essentially), and convert that to a regular string:

In [9]:
import base64

In [10]:
d_base64 = base64.b64encode(d_pickled)

In [11]:
d_base64

b'gASVFwAAAAAAAAB9lCiMAWGUSwGMAWKUSwKMAWOUSwN1Lg=='

Now we can take this byte string and convert it to a regular string:

In [12]:
d_string = d_base64.decode()

Now we can copy this string to our clipboard:

In [13]:
pyperclip.copy(d_string)

Of course, what's on the clipboard right now is not very useful unless you use Python to deserialize the object back to a Python object, which we can do as follows:

In [14]:
content = pyperclip.paste()
content

'gASVFwAAAAAAAAB9lCiMAWGUSwGMAWKUSwKMAWOUSwN1Lg=='

Which we can now deserialize this way:

In [15]:
new_d = pickle.loads(base64.b64decode(content.encode()))
new_d

{'a': 1, 'b': 2, 'c': 3}

We can write some small utility functions to do the serializing and deserializing:

In [16]:
def put_obj_to_clipboard(obj):
    pickled = pickle.dumps(obj)
    pyperclip.copy(base64.b64encode(pickled).decode())
    
def get_obj_from_clipboard():
    content = pyperclip.paste()
    return pickle.loads(base64.b64decode(content.encode()))

Let's test this out using a custom class:

In [17]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f"Person(name={self.name}, age={self.age})"

In [18]:
p = Person("John", 42)
p

Person(name=John, age=42)

In [19]:
put_obj_to_clipboard(p)

And the contents of the clipboard is:

In [20]:
pyperclip.paste()

'gASVNQAAAAAAAACMCF9fbWFpbl9flIwGUGVyc29ulJOUKYGUfZQojARuYW1llIwESm9obpSMA2FnZZRLKnViLg=='

Which can deserialize back to a `Person` object:

In [21]:
john = get_obj_from_clipboard()
john, john.name, john.age

(Person(name=John, age=42), 'John', 42)

One thing to note is that using `pickle` is considered "unsafe", i.e. only unpickle data you **trust**, as it is very easy to use pickle for various types of attacks.

One safer alternative, could be to use JSON encoding - with something like a dictionary here, it would be relatively simple to do using the `json` module, but you have to be aware that you may have to provide specialized serializers in order to serialize more complex data types such as dates, uuids, etc.

And there you have it - the `pyperclip` library - very simple to use, but restricted to text.